# Axisim Punch-Through Shear Test

This example couples two domains via an zero-thickness interface. 

State - 2021-07-12 [RC]:

- Slide34 included with not verified material parameters. The push through test is running for a small step size
- Change of the parameters to brittle response leads to convergence problems at the transition to inelastic state
- Either initial or secant stiffness has been used     

In [1]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
import warnings
import matplotlib.pylab as plt

In [2]:
from ibvpy.api import TStepBC
from ibvpy.bcond import BCSlice
from ibvpy.xmodel.xdomain_fe_grid_axisym import XDomainFEGridAxiSym
from ibvpy.xmodel.xdomain_interface import XDomainFEInterface

In [3]:
from ibvpy.fets import FETS2D4Q
from ibvpy.fets.fets1D5 import FETS1D52ULRH

In [4]:
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField

# Material models

In [5]:
from bmcs_matmod.slide.vslide_34 import Slide34
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic

# FE discretization

In [6]:
n_x_e = 10
n_inner_y_e = 5
n_outer_y_e = 5
L_x = 200.0
R_in = 100.0
R_out = 200.0
xd_inner = XDomainFEGridAxiSym(coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_x_e, n_inner_y_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
xd_outer = XDomainFEGridAxiSym(coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_x_e, n_outer_y_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
m1 = MATS3DElastic(E=28000, nu=0.3)
m2 = MATS3DElastic(E=28000, nu=0.3)

xd12 = XDomainFEInterface(
    I=xd_inner.mesh.I[1:-1, -1],
    J=xd_outer.mesh.I[1:-1, 0],
    fets=FETS1D52ULRH()
)

In [7]:
#bond_m = Slide34(E_T=10000, E_N=10000)
material_params = dict(
    E_T=1000, gamma_T=800, K_T=0, S_T=0.0005, c_T=1, bartau=3, 
    E_N=1000, S_N=0.001, c_N = 1, m = 0.01, f_t=3, f_c=20, f_c0=10, eta=0.0 # 0.5
)
bond_m = Slide34(**material_params)

In [8]:
fixed_y = BCSlice(slice=xd_inner.mesh[0, 0, 0, 0],
                 var='u', dims=[1], value=0)
fixed_outer_x = BCSlice(slice=xd_outer.mesh[0, :, 0, :],
                 var='u', dims=[0], value=0)
loaded_inner_x = BCSlice(slice=xd_inner.mesh[-1, :, -1, :],
                  var='u', dims=[0], value=-0.1)
free_inner_x = BCSlice(slice=xd_inner.mesh[0, :, 0, :],
                  var='u', dims=[0], value=0)
bc1 = [fixed_y, fixed_outer_x, loaded_inner_x]

m = TStepBC(
    domains=[(xd_inner, m1),
             (xd_outer, m2),
             (xd12, bond_m),
             ],
    bc=bc1,  # + bc2,
)

m.hist.vis_record = {
    'strain': Vis3DTensorField(var='eps_ab'),
    #        'damage': Vis3DStateField(var='omega_a'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 0.02
s.tstep.fe_domain.serialized_subdomains

In [9]:
bond_m.interact()

In [10]:
xd12.hidden = True
s.run()

t:	 0.00

<lambdifygenerated-7>:4: RuntimeWarning: invalid value encountered in true_divide
  return (array([[I*select([greater(O, (abs(sqrt((-X_x + tau_pi_x/(1 - omega_T))**2 + (-X_y + tau_pi_y/(1 - omega_T))**2)) - (Z + bartau)*(sig_pi/(-omega_N*Dummy_448 + 1) + f_t**2*m/(Z + bartau - 2*f_t*m))*(Z + bartau - 2*f_t*m)/(f_t**2*m))*sign(f_t)*sign(m)),greater(O, (abs(sqrt((-X_x + tau_pi_x/(1 - omega_T))**2 + (-X_y + tau_pi_y/(1 - omega_T))**2)) + (Z + bartau + f_c0*m)*(sig_pi/(-omega_N*Dummy_448 + 1) + f_c0 - m*(-f_c**2 + 2*f_c*f_c0 - f_c0**2)/(Z + bartau + 2*f_c*m - f_c0*m))*(Z + bartau + 2*f_c*m - f_c0*m)/(m*(-f_c**2 + 2*f_c*f_c0 - f_c0**2)))*sign(m)*sign(-f_c + f_c0)),True], [(-X_x + tau_pi_x/(1 - omega_T))*(f_t**2*m**2 - 2*f_t*m*(Z + bartau) + (Z + bartau)**2)**2/((1 - omega_T)*(Z + bartau)*sqrt(((-X_x + tau_pi_x/(1 - omega_T))**2 + (-X_y + tau_pi_y/(1 - omega_T))**2)*(f_t**2*m**2 - 2*f_t*m*(Z + bartau) + (Z + bartau)**2)**2/((Z + bartau)*(-Z - bartau + f_t*m)**2*(Z + bartau - 2*f_t*m)) + (sig

(0), 
	 0.02(1), 
	 0.04(5), 
	 0.06(5), 
	 0.08(5), 
	 0.10(5), 
	 0.12(5), 
	 0.14(5), 
	 0.16(5), 
	 0.18(5), 
	 0.20(5), 
	 0.22(5), 
	 0.24(5), 
	 0.26(5), 
	 0.28(5), 
	 0.30(5), 
	 0.32(5), 
	 0.34(5), 
	 0.36(5), 
	 0.38(5), 
	 0.40(5), 
	 0.42(5), 
	 0.44(5), 
	 0.46(5), 
	 0.48(5), 
	 0.50(5), 
	 0.52(5), 
	 0.54(5), 
	 0.56(5), 
	 0.58(5), 
	 0.60(5), 
	 0.62(5), 
	 0.64(5), 
	 0.66(5), 
	 0.68(5), 
	 0.70(5), 
	 0.72(5), 
	 0.74(5), 
	 0.76(5), 
	 0.78(5), 
	 0.80(5), 
	 0.82(5), 
	 0.84(5), 
	 0.86(5), 
	 0.88(5), 
	 0.90(5), 
	 0.92(5), 
	 0.94(5), 
	 0.96(5), 
	 0.98(5), 
	 1.00(5), 



# Plot the punch-through curve

In [11]:
F_to = m.hist.F_t
U_to = m.hist.U_t

In [12]:
F_to.shape

(51, 264)

In [13]:
F_l_t = np.sum(F_to[:, fixed_outer_x.dofs], axis=-1)
F_r_t = np.sum(F_to[:, loaded_inner_x.dofs], axis=-1)
U_r_t = np.average(U_to[:, loaded_inner_x.dofs], axis=-1)
U_l_t = np.average(U_to[:, free_inner_x.dofs], axis=-1)

In [14]:
_, ax = plt.subplots(1,1)
ax.plot(-U_r_t, -F_r_t)
ax.plot(-U_r_t, F_l_t)
ax.plot(-U_l_t, F_l_t)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
mlab.options.backend = 'envisage'
mlab.options.offscreen = False # 'envisage'
f_strain = mlab.figure()
scene = mlab.get_engine().scenes[-1]
scene.name = 'strain'
strain_viz = Viz3DTensorField(vis3d=m.hist['strain'])
strain_viz.setup()
strain_viz.warp_vector.filter.scale_factor = 0.1
strain_viz.plot(s.tstep.t_n)
mlab.show()